# Exploratory Data Analysis (EDA) - CRISP-DM Template
# {{ cookiecutter.project_name }}

**CRISP-DM Phases 1-2: Business Understanding & Data Understanding**

---

## Document Control

| | |
|---|---|
| **Project** | {{ cookiecutter.project_name }} |
| **Author** | {{ cookiecutter.author_name }} |
| **Contact** | {{ cookiecutter.email }} |
| **Date** | YYYY-MM-DD |
| **Version** | 1.0 |

---

## Table of Contents

1. [Business Understanding](#1-business-understanding)
2. [Data Understanding](#2-data-understanding)
3. [Data Quality Assessment](#3-data-quality-assessment)
4. [Exploratory Analysis](#4-exploratory-analysis)
5. [Statistical Analysis](#5-statistical-analysis)
6. [Feature Analysis](#6-feature-analysis)
7. [Hypothesis Testing](#7-hypothesis-testing)
8. [Initial Insights](#8-initial-insights)
9. [Next Steps](#9-next-steps)

## Setup

In [ ]:
# Standard libraries
import os
import sys
import warnings
from pathlib import Path
from datetime import datetime, timedelta

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, ks_2samp

# Data profiling
from ydata_profiling import ProfileReport

# Data validation
import pandera as pa
from pandera import Column, DataFrameSchema

# Project modules
sys.path.append(str(Path.cwd().parent.parent))
from src.data.aws_integration import get_s3_client, get_athena_client, get_iceberg_manager
from src.utils.logger import get_logger

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Logger
logger = get_logger(__name__)

print("Setup completo!")
print(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

---

## 1. Business Understanding

**CRISP-DM Phase 1**: Entender os objetivos e requisitos de negócio

### 1.1 Business Objectives

**Problema de Negócio**:
- [Descreva o problema de negócio que estamos tentando resolver]
- [Exemplo: Reduzir churn de clientes em 20%]

**Objetivos SMART**:
1. **Specific**: [Objetivo específico]
2. **Measurable**: [Como será medido - KPI]
3. **Achievable**: [É alcançável? Baseado em quê?]
4. **Relevant**: [Por que é relevante para o negócio?]
5. **Time-bound**: [Prazo - quando deve ser alcançado?]

**Success Criteria**:
- Métrica de Negócio: [Ex: Redução de 20% no churn]
- Métrica de ML: [Ex: Precision > 0.80, Recall > 0.70]
- ROI Esperado: [Ex: R$ X milhões/ano]

### 1.2 Business Questions

**Perguntas principais que queremos responder**:

1. [Pergunta 1: Ex: Quais características mais diferenciam clientes que churn vs não-churn?]
2. [Pergunta 2: Ex: Existe sazonalidade no churn?]
3. [Pergunta 3: Ex: Qual o perfil de cliente com maior risco?]
4. [Pergunta 4: Ex: Quanto tempo antes podemos prever o churn?]
5. [Pergunta 5: Ex: Quais ações de retenção são mais efetivas?]

### 1.3 ML Problem Definition

**Tradução para problema de ML**:

- **Problem Type**: [Classification / Regression / Clustering / Ranking]
- **Target Variable**: [Nome da variável alvo e definição]
- **Prediction Horizon**: [Ex: Prever churn nos próximos 30 dias]
- **Features**: [Tipos de features que usaremos - demográficas, comportamentais, transacionais]
- **Model Constraints**: [Latência < 100ms, Explicabilidade necessária, etc]

---

## 2. Data Understanding

**CRISP-DM Phase 2**: Coletar, descrever e explorar os dados

### 2.1 Data Collection

In [ ]:
# Configuração de acesso aos dados
DATA_SOURCE = "s3"  # s3 / athena / iceberg / local
S3_PATH = "s3://bucket-name/path/to/data.parquet"
ATHENA_QUERY = "SELECT * FROM database.table WHERE date >= '2024-01-01'"
LOCAL_PATH = "data/raw/sample_data.csv"

# Período de análise
START_DATE = "2024-01-01"
END_DATE = "2024-12-31"

In [ ]:
# Carregar dados
logger.info(f"Carregando dados de {DATA_SOURCE}...")

if DATA_SOURCE == "s3":
    s3_client = get_s3_client()
    df = s3_client.read_parquet(s3_key=S3_PATH.replace("s3://", "").split("/", 1)[1])
    
elif DATA_SOURCE == "athena":
    athena_client = get_athena_client()
    df = athena_client.execute_query(ATHENA_QUERY)
    
elif DATA_SOURCE == "iceberg":
    iceberg = get_iceberg_manager()
    df = iceberg.read_table(table_name="feature_store")
    
else:  # local
    df = pd.read_csv(LOCAL_PATH)

logger.info(f"Dados carregados: {df.shape[0]:,} linhas, {df.shape[1]} colunas")
print(f"\n📊 Dataset carregado com sucesso!")
print(f"   Shape: {df.shape}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

### 2.2 Data Description

In [ ]:
# Overview geral
print("=" * 80)
print("DATA OVERVIEW")
print("=" * 80)
df.info()

In [ ]:
# Primeiras linhas
print("\nPrimeiras 5 linhas:")
display(df.head())

In [ ]:
# Estatísticas descritivas
print("\nEstatísticas Descritivas (Features Numéricas):")
display(df.describe())

In [ ]:
# Estatísticas descritivas (categóricas)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
if len(categorical_cols) > 0:
    print("\nEstatísticas Descritivas (Features Categóricas):")
    display(df[categorical_cols].describe())

### 2.3 Data Dictionary

**Documentar cada feature**:

| Feature | Type | Description | Business Meaning | Source |
|---------|------|-------------|------------------|--------|
| feature_1 | int | [Descrição técnica] | [O que significa para o negócio] | [Tabela origem] |
| feature_2 | float | ... | ... | ... |
| target | binary | ... | ... | ... |

*Preencha esta tabela com todas as features do dataset*

### 2.4 Automated Data Profiling

**Gerar relatório automático com ydata-profiling**

In [ ]:
# Gerar profile report (pode demorar para datasets grandes)
# Descomente para executar

# profile = ProfileReport(
#     df,
#     title="{{ cookiecutter.project_name }} - Data Profiling Report",
#     explorative=True,
#     correlations={
#         "pearson": {"calculate": True},
#         "spearman": {"calculate": True},
#         "kendall": {"calculate": False},
#         "phi_k": {"calculate": False},
#     },
#     missing_diagrams={
#         "matrix": True,
#         "bar": True,
#         "heatmap": True,
#     },
# )

# # Salvar relatório
# output_path = "reports/data_profiling_report.html"
# profile.to_file(output_path)
# print(f"Relatório salvo em: {output_path}")

# # Exibir no notebook
# profile.to_notebook_iframe()

---

## 3. Data Quality Assessment

### 3.1 Missing Values Analysis

In [ ]:
# Análise de valores faltantes
missing_stats = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df) * 100).round(2),
    'Dtype': df.dtypes,
    'Unique_Values': df.nunique(),
    'Sample_Values': [df[col].dropna().head(3).tolist() for col in df.columns]
})

missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values(
    'Missing_Percent', ascending=False
)

print("\n" + "="*80)
print("MISSING VALUES ANALYSIS")
print("="*80)
if len(missing_stats) > 0:
    display(missing_stats)
else:
    print("✓ Nenhum valor faltante encontrado!")

In [ ]:
# Visualizar padrão de missing values
if len(missing_stats) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar plot
    missing_stats.sort_values('Missing_Percent')['Missing_Percent'].plot(
        kind='barh', ax=axes[0], color='coral'
    )
    axes[0].set_xlabel('Missing Percentage')
    axes[0].set_title('Missing Values by Feature')
    axes[0].grid(axis='x', alpha=0.3)
    
    # Heatmap (sample)
    missing_cols = missing_stats.head(20).index.tolist()
    sns.heatmap(
        df[missing_cols].isnull().head(100),
        cbar=False,
        yticklabels=False,
        cmap='RdYlGn_r',
        ax=axes[1]
    )
    axes[1].set_title('Missing Values Pattern (first 100 rows)')
    
    plt.tight_layout()
    plt.show()

### 3.2 Duplicate Analysis

In [ ]:
# Verificar duplicatas
n_duplicates = df.duplicated().sum()
duplicate_pct = (n_duplicates / len(df) * 100)

print("\n" + "="*80)
print("DUPLICATE ANALYSIS")
print("="*80)
print(f"Total duplicates: {n_duplicates:,} ({duplicate_pct:.2f}%)")

if n_duplicates > 0:
    print("\nExemplo de duplicatas:")
    display(df[df.duplicated(keep=False)].head(10))

### 3.3 Data Types Validation

In [ ]:
# Verificar tipos de dados
print("\n" + "="*80)
print("DATA TYPES DISTRIBUTION")
print("="*80)

dtype_counts = df.dtypes.value_counts()
print("\nContagem por tipo:")
print(dtype_counts)

# Listar por categoria
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()

print(f"\nNumeric columns ({len(numeric_cols)}): {numeric_cols[:5]}...")
print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols[:5]}...")
print(f"Datetime columns ({len(datetime_cols)}): {datetime_cols}")

### 3.4 Data Schema Validation

**Validação usando Pandera**

In [ ]:
# Definir schema esperado (exemplo)
# Customize de acordo com suas features

# schema = DataFrameSchema({
#     "customer_id": Column(str, nullable=False, unique=True),
#     "age": Column(int, checks=pa.Check.in_range(min_value=18, max_value=120)),
#     "income": Column(float, checks=pa.Check.greater_than(0)),
#     "target": Column(int, checks=pa.Check.isin([0, 1])),
# })

# # Validar
# try:
#     validated_df = schema.validate(df)
#     print("✓ Schema validation passed!")
# except pa.errors.SchemaError as e:
#     print("✗ Schema validation failed!")
#     print(e)

### 3.5 Data Quality Summary

In [ ]:
# Resumo de qualidade
quality_metrics = {
    'Total Records': len(df),
    'Total Features': len(df.columns),
    'Completeness': f"{((1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100):.2f}%",
    'Duplicates': f"{duplicate_pct:.2f}%",
    'Memory Usage': f"{df.memory_usage(deep=True).sum() / 1024**2:.2f} MB",
    'Numeric Features': len(numeric_cols),
    'Categorical Features': len(categorical_cols),
    'Datetime Features': len(datetime_cols),
}

print("\n" + "="*80)
print("DATA QUALITY SUMMARY")
print("="*80)
for metric, value in quality_metrics.items():
    print(f"{metric:.<30} {value:>20}")

---

## 4. Exploratory Analysis

### 4.1 Target Variable Analysis

In [ ]:
# Defina sua variável target
TARGET_COL = "target"  # Altere para sua variável target

if TARGET_COL in df.columns:
    print("\n" + "="*80)
    print(f"TARGET VARIABLE ANALYSIS: {TARGET_COL}")
    print("="*80)
    
    # Distribuição
    target_dist = df[TARGET_COL].value_counts()
    target_pct = df[TARGET_COL].value_counts(normalize=True) * 100
    
    print("\nDistribuição:")
    for val in target_dist.index:
        print(f"  {val}: {target_dist[val]:,} ({target_pct[val]:.2f}%)")
    
    # Visualizar
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    target_dist.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
    axes[0].set_title(f'Target Distribution: {TARGET_COL}')
    axes[0].set_ylabel('Count')
    axes[0].set_xlabel('')
    
    # Pie chart
    axes[1].pie(
        target_dist.values,
        labels=target_dist.index,
        autopct='%1.1f%%',
        colors=['#2ecc71', '#e74c3c'],
        startangle=90
    )
    axes[1].set_title('Target Proportion')
    
    plt.tight_layout()
    plt.show()
    
    # Verificar desbalanceamento
    imbalance_ratio = target_dist.max() / target_dist.min()
    print(f"\nImbalance Ratio: {imbalance_ratio:.2f}:1")
    if imbalance_ratio > 3:
        print("⚠️  Dataset desbalanceado! Considere técnicas de resampling.")
else:
    print(f"⚠️  Coluna '{TARGET_COL}' não encontrada no dataset")

### 4.2 Numeric Features Distribution

In [ ]:
# Distribuições de features numéricas
if len(numeric_cols) > 0:
    print("\n" + "="*80)
    print("NUMERIC FEATURES DISTRIBUTION")
    print("="*80)
    
    # Selecionar top N features para visualizar
    n_features = min(12, len(numeric_cols))
    features_to_plot = numeric_cols[:n_features]
    
    # Histogramas
    n_cols = 4
    n_rows = (n_features + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
    axes = axes.flatten()
    
    for idx, col in enumerate(features_to_plot):
        df[col].hist(bins=50, ax=axes[idx], edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'{col}\nMean: {df[col].mean():.2f}, Std: {df[col].std():.2f}')
        axes[idx].set_ylabel('Frequency')
    
    # Esconder subplots vazios
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Box plots para detectar outliers
if len(numeric_cols) > 0:
    print("\nOutlier Detection (Box Plots):")
    
    n_features = min(12, len(numeric_cols))
    features_to_plot = numeric_cols[:n_features]
    
    n_cols = 4
    n_rows = (n_features + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
    axes = axes.flatten()
    
    for idx, col in enumerate(features_to_plot):
        df.boxplot(column=col, ax=axes[idx])
        axes[idx].set_title(col)
        
        # Calcular outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()
        outlier_pct = (outliers / len(df) * 100)
        axes[idx].set_xlabel(f'Outliers: {outliers} ({outlier_pct:.1f}%)')
    
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

### 4.3 Categorical Features Analysis

In [ ]:
# Análise de features categóricas
if len(categorical_cols) > 0:
    print("\n" + "="*80)
    print("CATEGORICAL FEATURES ANALYSIS")
    print("="*80)
    
    for col in categorical_cols[:10]:  # Top 10
        print(f"\n{col}:")
        print(f"  Unique values: {df[col].nunique()}")
        print(f"  Top 5 values:")
        value_counts = df[col].value_counts().head(5)
        for val, count in value_counts.items():
            pct = (count / len(df) * 100)
            print(f"    {val}: {count:,} ({pct:.2f}%)")

In [ ]:
# Visualizar categóricas
if len(categorical_cols) > 0:
    # Selecionar features com cardinalidade razoável
    plottable_cats = [col for col in categorical_cols if df[col].nunique() <= 20]
    n_features = min(8, len(plottable_cats))
    features_to_plot = plottable_cats[:n_features]
    
    if len(features_to_plot) > 0:
        n_cols = 2
        n_rows = (n_features + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
        axes = axes.flatten() if n_features > 1 else [axes]
        
        for idx, col in enumerate(features_to_plot):
            df[col].value_counts().head(10).plot(kind='barh', ax=axes[idx])
            axes[idx].set_title(f'{col} (Top 10)')
            axes[idx].set_xlabel('Count')
        
        for idx in range(n_features, len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()

### 4.4 Temporal Analysis

*Se houver features temporais*

In [ ]:
# Análise temporal (se aplicável)
if len(datetime_cols) > 0:
    print("\n" + "="*80)
    print("TEMPORAL ANALYSIS")
    print("="*80)
    
    for date_col in datetime_cols:
        print(f"\n{date_col}:")
        print(f"  Min date: {df[date_col].min()}")
        print(f"  Max date: {df[date_col].max()}")
        print(f"  Date range: {(df[date_col].max() - df[date_col].min()).days} days")
        
        # Time series plot
        fig, ax = plt.subplots(figsize=(14, 5))
        df.groupby(df[date_col].dt.to_period('D')).size().plot(ax=ax)
        ax.set_title(f'Records Over Time: {date_col}')
        ax.set_ylabel('Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

---

## 5. Statistical Analysis

### 5.1 Correlation Analysis

In [ ]:
# Matriz de correlação
if len(numeric_cols) > 1:
    print("\n" + "="*80)
    print("CORRELATION ANALYSIS")
    print("="*80)
    
    # Calcular correlação
    corr_matrix = df[numeric_cols].corr()
    
    # Heatmap
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(
        corr_matrix,
        mask=mask,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        center=0,
        square=True,
        linewidths=1,
        cbar_kws={"shrink": 0.8}
    )
    plt.title('Correlation Matrix (Numeric Features)')
    plt.tight_layout()
    plt.show()
    
    # Top correlações
    print("\nTop 10 correlações mais fortes (|r| > 0.5):")
    corr_pairs = corr_matrix.unstack()
    corr_pairs = corr_pairs[corr_pairs != 1.0]  # Remove self-correlation
    corr_pairs = corr_pairs.sort_values(ascending=False)
    corr_pairs = corr_pairs[abs(corr_pairs) > 0.5].head(10)
    
    for (feat1, feat2), corr_val in corr_pairs.items():
        print(f"  {feat1} <-> {feat2}: r = {corr_val:.3f}")

### 5.2 Feature Importance (Target Correlation)

In [ ]:
# Correlação com target
if TARGET_COL in df.columns and df[TARGET_COL].dtype in [np.int64, np.float64]:
    print("\n" + "="*80)
    print("FEATURE IMPORTANCE (Correlation with Target)")
    print("="*80)
    
    # Correlação de features numéricas com target
    target_corr = df[numeric_cols + [TARGET_COL]].corr()[TARGET_COL].sort_values(ascending=False)
    target_corr = target_corr.drop(TARGET_COL)  # Remove target self-correlation
    
    print("\nTop 15 features correlacionadas com target:")
    print(target_corr.head(15))
    
    # Visualizar
    fig, ax = plt.subplots(figsize=(10, 8))
    target_corr.head(20).plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title(f'Top 20 Features Correlated with {TARGET_COL}')
    ax.set_xlabel('Correlation Coefficient')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
    plt.tight_layout()
    plt.show()

### 5.3 Normality Tests

In [ ]:
# Testar normalidade (Shapiro-Wilk para amostras pequenas, Kolmogorov-Smirnov para grandes)
if len(numeric_cols) > 0:
    print("\n" + "="*80)
    print("NORMALITY TESTS")
    print("="*80)
    
    normality_results = []
    
    for col in numeric_cols[:20]:  # Testar top 20
        # Remove NaN
        data = df[col].dropna()
        
        if len(data) < 5000:
            stat, p_value = stats.shapiro(data.sample(min(5000, len(data))))
            test_name = "Shapiro-Wilk"
        else:
            stat, p_value = stats.kstest(data, 'norm')
            test_name = "Kolmogorov-Smirnov"
        
        is_normal = "Yes" if p_value > 0.05 else "No"
        normality_results.append({
            'Feature': col,
            'Test': test_name,
            'Statistic': stat,
            'P-value': p_value,
            'Normal (α=0.05)': is_normal
        })
    
    normality_df = pd.DataFrame(normality_results)
    display(normality_df.sort_values('P-value', ascending=False))

---

## 6. Feature Analysis

### 6.1 Univariate Analysis by Target

*Analisar como cada feature se comporta nos diferentes valores de target*

In [ ]:
# Análise univariada por target (features numéricas)
if TARGET_COL in df.columns and len(numeric_cols) > 0:
    print("\n" + "="*80)
    print("UNIVARIATE ANALYSIS BY TARGET (Numeric)")
    print("="*80)
    
    # Selecionar top features
    n_features = min(8, len(numeric_cols))
    features_to_plot = numeric_cols[:n_features]
    
    n_cols = 2
    n_rows = (n_features + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
    axes = axes.flatten() if n_features > 1 else [axes]
    
    for idx, col in enumerate(features_to_plot):
        # Box plot por target
        df.boxplot(column=col, by=TARGET_COL, ax=axes[idx])
        axes[idx].set_title(f'{col} by {TARGET_COL}')
        axes[idx].set_xlabel(TARGET_COL)
        
        # Adicionar média
        means = df.groupby(TARGET_COL)[col].mean()
        for target_val, mean_val in means.items():
            axes[idx].text(
                target_val + 1, mean_val,
                f'μ={mean_val:.2f}',
                ha='center', va='bottom'
            )
    
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('')  # Remove auto title
    plt.tight_layout()
    plt.show()

In [ ]:
# Análise univariada por target (features categóricas)
if TARGET_COL in df.columns and len(categorical_cols) > 0:
    print("\n" + "="*80)
    print("UNIVARIATE ANALYSIS BY TARGET (Categorical)")
    print("="*80)
    
    # Selecionar categóricas com baixa cardinalidade
    plottable_cats = [col for col in categorical_cols if df[col].nunique() <= 10]
    n_features = min(6, len(plottable_cats))
    features_to_plot = plottable_cats[:n_features]
    
    if len(features_to_plot) > 0:
        n_cols = 2
        n_rows = (n_features + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
        axes = axes.flatten() if n_features > 1 else [axes]
        
        for idx, col in enumerate(features_to_plot):
            # Crosstab
            ct = pd.crosstab(df[col], df[TARGET_COL], normalize='index') * 100
            ct.plot(kind='bar', ax=axes[idx], stacked=False)
            axes[idx].set_title(f'{col} vs {TARGET_COL}')
            axes[idx].set_ylabel('Percentage')
            axes[idx].legend(title=TARGET_COL)
            axes[idx].set_xlabel('')
        
        for idx in range(n_features, len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()

### 6.2 Bivariate Analysis

In [ ]:
# Scatter plots bivariados (top features vs target)
if TARGET_COL in df.columns and len(numeric_cols) >= 2:
    print("\n" + "="*80)
    print("BIVARIATE ANALYSIS")
    print("="*80)
    
    # Selecionar top 6 features mais correlacionadas com target
    if df[TARGET_COL].dtype in [np.int64, np.float64]:
        top_features = df[numeric_cols + [TARGET_COL]].corr()[TARGET_COL].abs().sort_values(ascending=False)
        top_features = top_features.drop(TARGET_COL).head(6).index.tolist()
    else:
        top_features = numeric_cols[:6]
    
    if len(top_features) >= 2:
        # Criar pairplot
        sample_df = df[top_features + [TARGET_COL]].sample(
            min(5000, len(df)), random_state=42
        )
        
        pairplot = sns.pairplot(
            sample_df,
            hue=TARGET_COL,
            diag_kind='kde',
            plot_kws={'alpha': 0.6},
            height=3
        )
        pairplot.fig.suptitle('Pairplot - Top Features vs Target', y=1.01)
        plt.show()

### 6.3 Feature Engineering Ideas

*Documentar ideias para feature engineering baseadas na EDA*

**Ideias de Feature Engineering** (baseadas na análise acima):

1. **Interaction Features**:
   - [Ex: feature_1 * feature_2 - se mostraram correlação interessante]
   - [Ex: ratio de feature_3 / feature_4]

2. **Aggregations**:
   - [Ex: Média de transações por cliente]
   - [Ex: Total de compras nos últimos 30/60/90 dias]

3. **Binning/Discretization**:
   - [Ex: Idade em faixas: 18-25, 26-35, etc]
   - [Ex: Income em quartis]

4. **Temporal Features**:
   - [Ex: Day of week, month, quarter]
   - [Ex: Days since last purchase]
   - [Ex: Recency, Frequency, Monetary (RFM)]

5. **Encoding Strategies**:
   - [Ex: Target encoding para categóricas de alta cardinalidade]
   - [Ex: One-hot encoding para baixa cardinalidade]

6. **Missing Value Indicators**:
   - [Ex: is_missing_feature_X (binary flag)]

*Documente aqui suas ideias específicas baseadas nos dados*

---

## 7. Hypothesis Testing

### 7.1 Statistical Tests

In [ ]:
# Testes estatísticos para validar hipóteses
print("\n" + "="*80)
print("HYPOTHESIS TESTING")
print("="*80)

# Exemplo: Testar se features numéricas diferem entre classes do target
if TARGET_COL in df.columns and len(numeric_cols) > 0:
    target_classes = df[TARGET_COL].unique()
    
    if len(target_classes) == 2:
        print("\nMann-Whitney U Test (diferenças entre grupos):")
        print("H0: Distribuições são iguais entre classes")
        print("H1: Distribuições são diferentes\n")
        
        test_results = []
        
        for col in numeric_cols[:10]:  # Top 10
            group1 = df[df[TARGET_COL] == target_classes[0]][col].dropna()
            group2 = df[df[TARGET_COL] == target_classes[1]][col].dropna()
            
            if len(group1) > 0 and len(group2) > 0:
                stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
                
                test_results.append({
                    'Feature': col,
                    'Statistic': stat,
                    'P-value': p_value,
                    'Significant (α=0.05)': 'Yes' if p_value < 0.05 else 'No',
                    'Mean Group 0': group1.mean(),
                    'Mean Group 1': group2.mean(),
                })
        
        results_df = pd.DataFrame(test_results)
        display(results_df.sort_values('P-value'))

In [ ]:
# Chi-square test para features categóricas vs target
if TARGET_COL in df.columns and len(categorical_cols) > 0:
    print("\n" + "="*80)
    print("Chi-Square Test (Categorical Features vs Target)")
    print("="*80)
    print("H0: Features são independentes do target")
    print("H1: Features são dependentes do target\n")
    
    chi2_results = []
    
    for col in categorical_cols[:10]:
        if df[col].nunique() < 50:  # Evitar alta cardinalidade
            contingency_table = pd.crosstab(df[col], df[TARGET_COL])
            chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            
            chi2_results.append({
                'Feature': col,
                'Chi2': chi2,
                'P-value': p_value,
                'DOF': dof,
                'Significant (α=0.05)': 'Yes' if p_value < 0.05 else 'No'
            })
    
    if len(chi2_results) > 0:
        chi2_df = pd.DataFrame(chi2_results)
        display(chi2_df.sort_values('P-value'))

### 7.2 Business Hypotheses

*Testar hipóteses específicas do negócio*

**Hipóteses de Negócio para Testar**:

1. **H1**: [Exemplo: Clientes premium têm menor taxa de churn que clientes regulares]
   - Teste: [Mann-Whitney / Chi-square]
   - Resultado: [Aceitar/Rejeitar H0]
   - Implicação: [O que isso significa para o negócio]

2. **H2**: [Exemplo: Clientes que usaram suporte nos últimos 30 dias têm maior churn]
   - Teste: [...]
   - Resultado: [...]
   - Implicação: [...]

3. **H3**: [Sua hipótese]
   - ...

*Adicione código cells abaixo para testar cada hipótese*

In [ ]:
# Testar H1
# [Seu código aqui]

In [ ]:
# Testar H2
# [Seu código aqui]

---

## 8. Initial Insights

### 8.1 Key Findings

**Principais descobertas da EDA**:

1. **Data Quality**:
   - [Ex: Dataset tem 95% de completeness, apenas 3 features com >10% missing]
   - [Ex: 5% de duplicatas identificadas, necessário tratamento]
   - [Ex: Outliers significativos em feature X (20% dos dados)]

2. **Target Variable**:
   - [Ex: Target desbalanceado 80:20, necessário resampling]
   - [Ex: Distribuição temporal estável, sem mudanças bruscas]

3. **Feature Patterns**:
   - [Ex: Top 3 features correlacionadas com target: A, B, C]
   - [Ex: Feature X tem correlação forte (0.7) com target]
   - [Ex: Features Y e Z são altamente correlacionadas (0.9), considerar remover uma]

4. **Business Insights**:
   - [Ex: Clientes premium têm 50% menos churn que regulares (p<0.01)]
   - [Ex: Sazonalidade identificada: maior churn em dezembro]
   - [Ex: Clientes com >5 transações/mês raramente churnam]

5. **Data Issues**:
   - [Ex: Feature idade tem valores negativos - necessário validação]
   - [Ex: 30% dos clientes sem informação de segmento]
   - [Ex: Dados antes de 2023 parecem incompletos]

### 8.2 Recommendations

**Recomendações para próximas fases**:

**Data Preparation (CRISP-DM Phase 3)**:
1. [Ex: Tratar missing values com forward fill para features temporais]
2. [Ex: Remover duplicatas usando customer_id + date como chave]
3. [Ex: Aplicar log transform em features com alta skewness]
4. [Ex: Criar feature 'dias_desde_ultima_compra' para capturar recência]
5. [Ex: Aplicar SMOTE para balancear target]

**Modeling (CRISP-DM Phase 4)**:
1. [Ex: Testar Random Forest, XGBoost e LightGBM]
2. [Ex: Usar time-series split para validação (não random split)]
3. [Ex: Feature selection: remover features com correlação >0.95]
4. [Ex: Otimizar threshold de decisão para maximizar recall (catch churners)]

**Data Collection**:
1. [Ex: Solicitar dados de suporte (tickets, interações)]
2. [Ex: Integrar dados de campanhas de marketing]
3. [Ex: Adicionar features de produto (uso de features premium)]

---

## 9. Next Steps

### 9.1 Action Items

**Ações imediatas**:

- [ ] **Data Cleaning**:
  - [ ] Tratar missing values
  - [ ] Remover duplicatas
  - [ ] Validar e corrigir outliers
  
- [ ] **Feature Engineering**:
  - [ ] Criar features temporais (RFM)
  - [ ] Criar interaction features
  - [ ] Aplicar encoding em categóricas
  
- [ ] **Data Pipeline**:
  - [ ] Implementar pipeline de transformação em `src/pipelines/DS/feature_pipeline.py`
  - [ ] Documentar transformações no Data Card
  - [ ] Configurar testes de qualidade de dados
  
- [ ] **Documentation**:
  - [ ] Completar Data Card com estatísticas da EDA
  - [ ] Documentar business insights encontrados
  - [ ] Atualizar Business Requirements com validações
  
- [ ] **Modeling**:
  - [ ] Preparar dataset limpo para modelagem
  - [ ] Definir estratégia de validação (time-series split)
  - [ ] Iniciar baseline model

### 9.2 Transition to Next Phase

**CRISP-DM Phase 3: Data Preparation**

Próximo notebook: `02_data_preparation.ipynb`

Este notebook deve:
1. Implementar todas as transformações identificadas nesta EDA
2. Criar versão limpa do dataset
3. Salvar em S3/Iceberg para uso em treinamento
4. Documentar todas as transformações aplicadas

---

## Summary Export

In [ ]:
# Exportar resumo da EDA para arquivo
summary = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_shape': df.shape,
    'total_features': len(df.columns),
    'numeric_features': len(numeric_cols),
    'categorical_features': len(categorical_cols),
    'datetime_features': len(datetime_cols),
    'completeness_pct': float(((1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100)),
    'duplicate_pct': float((df.duplicated().sum() / len(df) * 100)),
}

if TARGET_COL in df.columns:
    summary['target_column'] = TARGET_COL
    summary['target_distribution'] = df[TARGET_COL].value_counts().to_dict()

# Salvar como JSON
import json
output_path = 'reports/eda_summary.json'
os.makedirs('reports', exist_ok=True)
with open(output_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\nEDA Summary salvo em: {output_path}")
print(json.dumps(summary, indent=2))

---

## Document Control

**EDA Status**: [Draft / In Review / Approved]

**Reviewers**:
- [ ] Data Scientist
- [ ] Business Stakeholder
- [ ] Data Engineer

**Approval**: _________________________  Date: __________

**Next Review**: [YYYY-MM-DD]